# Model extraction -  geo data using open Ai

Step 1: PDF Loader & Page Windowing

Extracted all 257 pages of text from your thesis using pypdf.

Applied smart windowing rules: started at the first “Introduction” page (never before page 17–19).

Handled multiple references: when a “References/Bibliography” page is hit, the code pauses, looks ahead 4–6 pages—

If another section header (e.g., “Chapter IV”) is found, it skips the references and resumes extraction.

If not, it stops for good at the final references.

Created a clean subset (selected_pages_text) with only the relevant content for downstream extraction, excluding chapter-level reference lists and back-matter noise.

In [ ]:

!pip -q install pypdf

import re
from pathlib import Path
from pypdf import PdfReader

PDF_PATH = "/content/PhDThesis__Masurel2015.pdf"

assert PDF_PATH is not None and Path(PDF_PATH).exists(), "PDF_PATH not set or file not found."

# === Config ===
MIN_SKIP_PAGES = 17
MAX_SKIP_PAGES_FALLBACK = 19
TAIL_EXCLUDE_PAGES = 5
LOOKAHEAD_PAGES = 6  # how far to look ahead after "References"

INTRO_PATTERNS = [
    r'^\s*(chapter\s+\w+\.?\s+)?introduction\b',
    r'^\s*introduction\b',
]
STOP_HEADINGS = [
    r'^\s*references\b',
    r'^\s*bibliograph(y|ies)\b',
    r'^\s*acknowledg(e)?ments\b',
]
SECTION_HEADINGS = [
    r'^\s*chapter\s+[ivxlcdm]+\b',
    r'^\s*chapter\s+\d+\b',
    r'^\s*\d+\.\s+[A-Z]',
]

# === Helpers ===
def read_pdf_pages_text(pdf_path: str):
    reader = PdfReader(pdf_path)
    return [page.extract_text() or "" for page in reader.pages]

def match_any(text, patterns):
    for pat in patterns:
        if re.search(pat, text, flags=re.IGNORECASE | re.MULTILINE):
            return True
    return False

def first_intro_page(pages_text):
    for i in range(MIN_SKIP_PAGES, len(pages_text)):
        if match_any(pages_text[i][:2000], INTRO_PATTERNS):
            return i
    return MAX_SKIP_PAGES_FALLBACK

def smart_extract_window(pages_text):
    N = len(pages_text)
    start_idx = min(first_intro_page(pages_text), N-1)
    selected, skipping = [], False

    i = start_idx
    while i < N - TAIL_EXCLUDE_PAGES:
        head = (pages_text[i] or "")[:1500]

        if match_any(head, STOP_HEADINGS):
            # pause extraction at refs
            lookahead_text = " ".join(pages_text[i+1:i+LOOKAHEAD_PAGES+1])
            if match_any(lookahead_text, SECTION_HEADINGS):
                # skip this refs block, then resume after lookahead
                print(f"Skipped References block at page {i+1}, resuming later…")
                i += LOOKAHEAD_PAGES
                continue
            else:
                print(f"Final stop at References block on page {i+1}")
                break

        selected.append((i, pages_text[i]))
        i += 1

    return selected, start_idx

# === Run ===
pages_text = read_pdf_pages_text(PDF_PATH)
N = len(pages_text)
print(f"Loaded {N} pages from: {PDF_PATH}")

selected_pairs, start_idx = smart_extract_window(pages_text)
selected_pages_text = [t for _, t in selected_pairs]
selected_indices = [i for i, _ in selected_pairs]

print("\n=== SMART WINDOW SUMMARY ===")
print(f"Total pages: {N}")
print(f"Selected pages: {len(selected_pages_text)}")
print(f"From page {selected_indices[0]+1} to {selected_indices[-1]+1}")

# Peek at first and last selected page
print("\n--- Start page preview ---")
print(selected_pages_text[0][:500].replace("\n", " "))
print("\n--- End page preview ---")
print(selected_pages_text[-1][:500].replace("\n", " "))


Loaded 257 pages from: /content/PhDThesis__Masurel2015.pdf

=== SMART WINDOW SUMMARY ===
Total pages: 257
Selected pages: 234
From page 19 to 252

--- Start page preview ---
    1    Chapter I. Introduction     1. Preamble and knowledge gaps    Paleoproterozoic (i.e. Biri mian) volcano-plutonic belts and sedimentary basins  of West Africa not only  provide a complete record of crustal growth but also host a  number of world-class gold deposits (Abouc hami et al., 1990; Boher et al., 1992). To  date, a large number of studies have focused on the Baoulé-Mossi domain, which covers  portions of Burkina Faso, Côte d’Ivoire, Ghana, Guinea and Mali (Fig. 1). Gold  deposits

--- End page preview ---
    234    3. Future work    Despite the work undertaken by the candi date, a number of questions remain  unanswered, some of which are directly relevant to exploration targeting:     3.1. Source of fluids and metals    Recent research suggests that the divers ity in mineralisation styles and ore  

What the outputs are:

selected_pages_text → the clean list of 234 page texts (from page 19 through 252), excluding chapter-level references and the final back-matter references.

selected_page_offset → the original starting page index (18), so page numbers can always be re-attached correctly for provenance.

Preview logs → console prints confirming the chosen start page (Introduction, page 19) and end page (before the last references, page 252), with short text snippets for sanity checking.

# Step 2 (Block Extraction: sentences, tables, captions)

What the code is doing (3 points):

Splits each page into smaller chunks (paragraphs/lines) and classifies them as sentence, table_row, or caption.

Sentence tokenization: if a chunk is narrative text, it’s further split into atomic sentences.

Keeps provenance: every block is tagged with page_num and block_id for traceability.

In [ ]:

!pip -q install nltk

import re
import nltk
nltk.download("punkt", quiet=True)
nltk.download("punkt_tab", quiet=True) # Add this line

from nltk.tokenize import sent_tokenize
import pandas as pd

def detect_block_type(text):
    """Heuristic block classifier"""
    # Table-like if many numbers or tab spacing
    if re.search(r"\d", text) and (text.count(" ") > 10 or "\t" in text):
        return "table_row"
    # Caption if starts with Figure/Table
    if re.match(r"^\s*(figure|fig\.|table)\s+\d+", text, re.I):
        return "caption"
    return "sentence"

def extract_blocks(pages_text, page_indices):
    blocks = []
    for local_idx, page_text in enumerate(pages_text):
        page_num = page_indices[local_idx] + 1  # human page number
        # Split page into paragraphs/lines
        chunks = [c.strip() for c in page_text.split("\n") if c.strip()]
        block_id = 0
        for chunk in chunks:
            block_type = detect_block_type(chunk)
            # If sentence mode: split further
            if block_type == "sentence":
                sentences = sent_tokenize(chunk)
                for sent in sentences:
                    blocks.append({
                        "page_num": page_num,
                        "block_id": f"{page_num}-{block_id}",
                        "block_type": "sentence",
                        "text": sent
                    })
                    block_id += 1
            else:
                blocks.append({
                    "page_num": page_num,
                    "block_id": f"{page_num}-{block_id}",
                    "block_type": block_type,
                    "text": chunk
                })
                block_id += 1
    return pd.DataFrame(blocks)

# Run block extraction
df_blocks = extract_blocks(selected_pages_text, selected_indices)

print(f"Extracted {len(df_blocks)} blocks")
display(df_blocks.head(10)) # Use display for better output

Extracted 5825 blocks


,page_num,block_id,block_type,text
0,19,19-0,sentence,1
1,19,19-1,sentence,Chapter I.
2,19,19-2,sentence,Introduction
3,19,19-3,sentence,1.
4,19,19-4,sentence,Preamble and knowledge gaps
5,19,19-5,sentence,Paleoproterozoic (i.e.
6,19,19-6,sentence,Biri mian) volcano-plutonic belts and sediment...
7,19,19-7,sentence,of West Africa not only provide a complete re...
8,19,19-8,table_row,number of world-class gold deposits (Abouc ham...
9,19,19-9,sentence,"date, a large number of studies have focused o..."


output:

Split all 234 pages into 5825 atomic blocks (sentences, table-like rows, and captions).

Classified each block into sentence, table_row, or caption using simple heuristics.

Attached provenance (page_num, block_id) so each block can be traced back to its source page.

📄 What the output is:

df_blocks: a DataFrame with 4 key columns — page_num, block_id, block_type, and text.

First rows confirm: the start page (19) has its chapter heading, intro sentences, and even a table-like line flagged correctly.

# tep 3: Dictionary & Regex Pre-Filter

What this code does (3 points)

Loads a ready-to-use DICT (no commented placeholders) with core geo terms, units, and aliases/normalisers.

Scans each block for (a) at least one domain keyword and (b) quantitative patterns (numbers + units, ranges, ±error, oxide %).

Emits a shortlisted DataFrame with provenance + parsed numbers, and saves it to /content/candidate_blocks_step3.csv.

In [ ]:

!pip -q install pandas rapidfuzz

import re, json
from collections import Counter
from rapidfuzz import process, fuzz
import pandas as pd

# A) Dictionary
DICT = {
    "rock_type": [
        "limestone","dolomite","dolostone","wacke","arenite","siltstone","argillite",
        "mudstone","shale","sandstone","conglomerate","breccia","granite","monzogranite",
        "granodiorite","diorite","tonalite","gabbro","basalt","andesite","rhyolite",
        "komatiite","trachyte","phonolite","gneiss","schist","quartzite"
    ],
    "minerals": [
        "pyrite","arsenopyrite","chalcopyrite","pyrrhotite","sphalerite","galena",
        "magnetite","hematite","native gold","electrum","muscovite","biotite",
        "chlorite","sericite","albite","epidote","carbonate","quartz","feldspar"
    ],
    "mineralisation": [
        "sulfide","sulphide","stockwork","vein","disseminated","massive","breccia",
        "replacement","porphyry","orogenic"
    ],
    "tectonism_event": [
        "D1","D2","D3","D4","compressional","extensional","transcurrent",
        "sinistral","dextral","thrust","fold","shear zone","shear"
    ],
    "structures": [
        "foliation","lineation","cleavage","vein","veinlet","breccia","schistosity",
        "plunge","strike","dip","bedding","lamination","fault"
    ],
    "methods": [
        "ICP-MS","LA-ICP-MS","EPMA","SEM","XRD","XRF","AAS","fire assay",
        "TIMS","ID-TIMS","MC-ICP-MS","SIMS","LAICPMS","microprobe"
    ],
    "geochronology_terms": [
        "zircon","monazite","baddeleyite","concordia","intercept","weighted mean",
        "MSWD","discordant","206Pb/238U","207Pb/206Pb","U–Pb","U-Pb","age","dated"
    ],
    "assay_elements": [
        "Au","Ag","As","Sb","Cu","Pb","Zn","Ni","Co","Fe","S",
        "SiO2","Al2O3","MgO","CaO","K2O","Na2O","TiO2","P2O5","LOI","Cr2O3","MnO"
    ],
    "units": ["ppm","ppb","wt%","%","g/t","mg/kg","µg/g","ug/g","Ma","Ga","°C","deg C"],
    "stopwords_geo": ["references","bibliography","acknowledgements","appendix"]
}

UNIT_NORMALISE = {"gpt": "g/t", "percent": "%", "ug/g": "µg/g", "deg c": "°C"}
ALIAS_MAP = {
    "gold":"Au","arsenic":"As","antimony":"Sb","copper":"Cu","lead":"Pb","zinc":"Zn",
    "nickel":"Ni","cobalt":"Co","sulfur":"S","sulphur":"S"
}

def normalise_unit(u: str|None) -> str|None:
    if not u: return None
    u = u.strip().lower()
    return UNIT_NORMALISE.get(u, u).replace("ug/g","µg/g")

def alias_text(text: str) -> str:
    low = text.lower()
    for k, v in ALIAS_MAP.items():
        low = re.sub(rf"\b{k}\b", v.lower(), low)
    return low

# B) Numeric/Unit regex
NUM = r"\d+(?:[\.,]\d+)?"
RANGE_SEP = r"(?:–|-|to)"
PATTERNS = {
    "num_unit": re.compile(rf"(?P<val>{NUM})\s*(?P<unit>ppm|ppb|wt%|%|g\/t|mg\/kg|µg\/g|ug\/g|Ma|Ga|°C)\b", re.I),
    "range": re.compile(rf"(?P<v1>{NUM})\s*{RANGE_SEP}\s*(?P<v2>{NUM})\s*(?P<unit>ppm|ppb|%|g\/t|Ma|Ga)?\b", re.I),
    "between_age": re.compile(rf"\bbetween\s+(?P<v1>{NUM})\s*(?:–|-|to|and)\s*(?P<v2>{NUM})\s*(?P<unit>Ma|Ga)\b", re.I),
    "plusminus": re.compile(rf"(?P<mean>{NUM})\s*(?:±|\+\/-)\s*(?P<err>{NUM})\s*(?P<unit>Ma|g\/t|ppm|%|°C)?\b", re.I),
    "oxide_pct": re.compile(rf"(?P<oxide>[A-Z][a-z]?(?:\d)?O\d?)\s*(?P<val>{NUM})\s*%", re.I),
}

# C) Noise guards & context checks
MONTHS = r"(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec|january|february|march|april|june|july|august|september|october|november|december)"
CITATION_CUES = r"\b(journal|geological society|special publications|bulletin|v\.|vol\.|issue|no\.|pp?\.|doi:|issn|isbn|proceedings|symposium|conference|abstracts|et al\.)\b"

RANGE_CONTEXT_OK = [
    "ma","ga","age","dated","u–pb","u-pb","zircon","monazite","concordia","intercept",
    "ppm","ppb","g/t","wt%","%","oxide","sio2","feo","tio2","mgo","cao","k2o","na2o","p2o5",
    "mg#","mswd"
]

def has_range_context(text: str) -> bool:
    low = text.lower()
    return any(tok in low for tok in RANGE_CONTEXT_OK)

def looks_like_date_or_citation(text: str) -> bool:
    low = text.lower()
    if re.search(MONTHS, low) and re.search(r"\b\d{1,2}\s*(?:–|-|to)\s*\d{1,2}\b", low):
        return True
    if re.search(CITATION_CUES, low) and re.search(r"\b\d{2,6}\s*(?:–|-|to)\s*\d{2,6}\b", low):
        return True
    return False

def age_signal_present(text: str) -> bool:
    low = text.lower()
    return any(tok in low for tok in ["u–pb","u-pb","concordia","mswd","age","dated"])

# D) Candidate finder over df_blocks
def find_candidates_from_blocks(df_blocks, min_keywords=1, require_numeric=True, fuzzy_cutoff=95):
    recs = []
    for _, row in df_blocks.iterrows():
        raw_text = str(row["text"]).strip()
        if not raw_text:
            continue

        low = alias_text(raw_text)

        # Stronger citation/date guard: allow only if explicit unit/± OR clear age signal
        if looks_like_date_or_citation(raw_text):
            if not (PATTERNS["num_unit"].search(raw_text) or PATTERNS["plusminus"].search(raw_text) or age_signal_present(raw_text)):
                continue

        # Keyword categories
        keyword_hits = set()
        for cat, terms in DICT.items():
            if cat in ("units","stopwords_geo"):
                continue
            if any(t.lower() in low for t in terms):
                keyword_hits.add(cat); continue
            best = process.extractOne(low, terms, scorer=fuzz.partial_ratio, score_cutoff=fuzzy_cutoff)
            if best: keyword_hits.add(cat)

        if len(keyword_hits) < min_keywords:
            continue

        # Quantitative patterns
        m_numunit   = list(PATTERNS["num_unit"].finditer(raw_text))
        m_pm        = list(PATTERNS["plusminus"].finditer(raw_text))
        m_oxide     = list(PATTERNS["oxide_pct"].finditer(raw_text))
        m_range     = list(PATTERNS["range"].finditer(raw_text))
        m_between   = list(PATTERNS["between_age"].finditer(raw_text))

        explicit_ok = bool(m_numunit or m_pm or m_oxide)
        valid_numeric = explicit_ok

        # Allow ranges only with context; require stronger geo signal if still unit-less
        if not explicit_ok:
            if (m_range or m_between) and has_range_context(raw_text):
                valid_numeric = len(keyword_hits) >= 2
            else:
                valid_numeric = False

        # Co-occurrence rule for Ma/Ga without ± :
        if explicit_ok and not m_pm:
            # if ONLY Ma/Ga units found (no ppm/%/g/t/°C), demand geochron signal or ≥2 categories
            only_age_units = all((m.group("unit") or "").lower() in ["ma","ga"] for m in m_numunit) if m_numunit else False
            if only_age_units and not (("geochronology_terms" in keyword_hits) or (len(keyword_hits) >= 2)):
                valid_numeric = False

        if require_numeric and not valid_numeric:
            continue

        # Parse numbers
        numbers = []
        for m in m_numunit:
            numbers.append({
                "type":"num_unit",
                "value": float(m.group("val").replace(",", ".")),
                "unit":  normalise_unit(m.group("unit"))
            })
        for m in m_pm:
            numbers.append({
                "type":"plusminus",
                "mean":  float(m.group("mean").replace(",", ".")),
                "error": float(m.group("err").replace(",", ".")),
                "unit":  normalise_unit(m.group("unit"))
            })
        for m in m_oxide:
            numbers.append({
                "type":"oxide_pct",
                "oxide": m.group("oxide"),
                "value": float(m.group("val").replace(",", ".")),
                "unit":  "%"
            })
        # Age ranges
        if m_between:
            for m in m_between:
                numbers.append({
                    "type":"range",
                    "value_min": float(m.group("v1").replace(",", ".")),
                    "value_max": float(m.group("v2").replace(",", ".")),
                    "unit":      normalise_unit(m.group("unit"))
                })
        elif m_range and has_range_context(raw_text) and len(keyword_hits) >= 2:
            for m in m_range:
                numbers.append({
                    "type":"range",
                    "value_min": float(m.group("v1").replace(",", ".")),
                    "value_max": float(m.group("v2").replace(",", ".")),
                    "unit":      normalise_unit(m.group("unit")) if m.group("unit") else None
                })

        if not numbers:
            continue

        recs.append({
            "page_num": row["page_num"],
            "block_id": row["block_id"],
            "block_type": row["block_type"],
            "text": raw_text,
            "keyword_categories": sorted(list(keyword_hits)),
            "numbers_json": json.dumps(numbers, ensure_ascii=False),
            "has_numbers": True
        })

    return pd.DataFrame.from_records(recs)

#  E) Run and save
df_cands = find_candidates_from_blocks(df_blocks, min_keywords=1, require_numeric=True)

df_cands = df_cands.sort_values(["page_num","block_id"]).reset_index(drop=True)
print(f"Candidate blocks found: {len(df_cands)}")

display_cols = ["page_num","block_id","block_type","keyword_categories","text","numbers_json"]
print(df_cands[display_cols].head(12).to_string(index=False)[:2500])

OUT_PATH = "/content/candidate_blocks_step3.csv"
df_cands.to_csv(OUT_PATH, index=False)
print(f"\nSaved candidates → {OUT_PATH}")

# Quick category stats
cat_counts = Counter()
for cats in df_cands["keyword_categories"]:
    for c in cats:
        cat_counts[c] += 1
print("\nTop categories (rough):")
for c, n in cat_counts.most_common(12):
    print(f"  {c:22s} {n}")


Candidate blocks found: 174
 page_num block_id block_type                               keyword_categories                                                                                    text                                                                                                                                                                                                                               numbers_json
       19    19-18  table_row                                 [assay_elements]  2158 Ma gold at Wassa, Parra-Avila, in press; 2105 ± 2 Ma gold at Ashanti, Oberthür et                                                                 [{"type": "num_unit", "value": 2158.0, "unit": "ma"}, {"type": "num_unit", "value": 2.0, "unit": "ma"}, {"type": "plusminus", "mean": 2105.0, "error": 2.0, "unit": "ma"}]
       19    19-19  table_row                                 [assay_elements]     al., 1998; 2063 ± 9 Ma gold at Damang, Pigois et al., 2003). Nevertheless, the vast  

# Step 4: LLM Structured Extraction.


What this code will do (3 points)

Calls an LLM on your shortlisted df_cands blocks to extract strict JSON facts with categories, values, units, and provenance.

Validates/normalises the JSON (schema checks, unit normalisation, numeric coercion) and assigns a confidence.

Writes tidy outputs: a JSONL of raw model facts and a flattened CSV/Parquet ready for analysis.

⚙️ You’ll need an OpenAI-compatible API key in Colab:
import os; os.environ["OPENAI_API_KEY"] = "sk-..."

In [ ]:
#Step 4) LLM Structured Extraction
!pip -q install --no-cache-dir pandas pydantic requests

import os, json, time, re, getpass, requests
import pandas as pd
from typing import List, Optional, Dict, Any
from pydantic import BaseModel, Field, ValidationError


OPENAI_KEY = None
try:
    from google.colab import userdata
    OPENAI_KEY = userdata.get("sandra")
except Exception:
    OPENAI_KEY = None

if not OPENAI_KEY:
    OPENAI_KEY = os.environ.get("OPENAI_API_KEY")
if not OPENAI_KEY:
    print("Paste your OpenAI API key (hidden):")
    OPENAI_KEY = getpass.getpass("> ").strip()
if not OPENAI_KEY:
    raise RuntimeError("No API key provided (secret 'adhi.key' not found).")

os.environ["OPENAI_API_KEY"] = OPENAI_KEY


API_URL     = "https://api.openai.com/v1/chat/completions"
MODEL_NAME  = "gpt-4o-mini"
BATCH_SIZE  = 20
MAX_BLOCKS  = None
TEMPERATURE = 0
RETRY       = 3
SLEEP_BETWEEN = 1.25

ALLOWED_CATEGORIES = [
    "lithology","rock_type","minerals",
    "mineralisation","tectonism_event","structures",
    "assay","geochemistry","geochronology","magmatism",
    "methods","stratigraphy"
]
ALLOWED_UNITS = ["ppm","ppb","wt%","%","g/t","mg/kg","µg/g","Ma","Ga","°C"]

UNIT_NORMALISE = {"gpt": "g/t", "percent": "%", "ug/g": "µg/g", "deg c": "°C"}
def normalise_unit(u: Optional[str]) -> Optional[str]:
    if not u: return None
    u = u.strip()
    lu = UNIT_NORMALISE.get(u.lower(), u.lower())
    return {"ma":"Ma","ga":"Ga","°c":"°C"}.get(lu, lu)

# C) Output schema
class Fact(BaseModel):
    category: str = Field(..., description=f"One of: {ALLOWED_CATEGORIES}")
    entity: Optional[str] = None
    attribute: str = Field(..., description="e.g., Au, As, SiO2, age, MSWD, grade, orientation")
    value: Optional[float] = None
    unit: Optional[str] = None
    value_min: Optional[float] = None
    value_max: Optional[float] = None
    error: Optional[float] = None
    error_unit: Optional[str] = None
    method: Optional[str] = None
    mineral_phase: Optional[str] = None
    sample_id: Optional[str] = None
    context_snippet: str = Field(..., max_length=240)
    page: int = Field(..., description="1-based page number")
    block_id: str = Field(..., description="Step-2 block id")
    confidence: float = Field(..., ge=0.0, le=1.0)

class BatchOut(BaseModel):
    results: list[Fact] = Field(default_factory=list)

#D) Prompts
SYSTEM_PROMPT = """You extract structured geology facts from thesis text.
Return STRICT JSON matching the schema. Do not add keys.
Prefer atomic facts (split multi-values). Provide <=240 char context_snippet.
If no measurable geological quantity exists, return results: []."""

USER_TEMPLATE = """You will be given a list of blocks from a geology thesis with page numbers and block_ids.
For each block, extract zero or more atomic facts as JSON objects with this schema:

Fact: {{
  "category": one of ["lithology","rock_type","minerals","mineralisation","tectonism_event","structures","assay","geochemistry","geochronology","magmatism","methods","stratigraphy"],
  "entity": optional string,
  "attribute": string (e.g., Au, As, SiO2, age, MSWD, grade, orientation),
  "value": number (if scalar),
  "unit": optional string in ["ppm","ppb","wt%","%","g/t","mg/kg","µg/g","Ma","Ga","°C"],
  "value_min": number (if a range),
  "value_max": number (if a range),
  "error": number (if ± is present),
  "error_unit": optional string (often same as unit),
  "method": optional string (e.g., U–Pb, LA-ICP-MS, EPMA, Fire Assay),
  "mineral_phase": optional string (e.g., zircon, monazite),
  "sample_id": optional string,
  "context_snippet": string (<=240 chars),
  "page": integer (1-based),
  "block_id": string,
  "confidence": float between 0 and 1
}}

Rules:
- Use "geochronology" + attribute "age" for Ma/Ga ages; set 'error' if ±; set 'value_min/value_max' for ranges like "between 2115 and 2060 Ma".
- Use "assay" for element grades (Au, As, Zn...) with g/t, ppm, %, etc.; use "geochemistry" for oxides like SiO2, Al2O3.
- Do not invent sample IDs or methods; include only if present or clearly implied.
- Ignore citation-only lines.

Return strictly: {{"results":[...]}}.

Blocks:
{blocks_json}
"""

#Payload builder
def make_blocks_payload(df: pd.DataFrame) -> List[Dict[str, Any]]:
    page_col = "page" if "page" in df.columns else "page_num"
    return [
        {"page": int(row[page_col]), "block_id": str(row["block_id"]), "text": str(row["text"])}
        for _, row in df.iterrows()
    ]

# F) LLM call (direct REST)
def call_llm(blocks_chunk: List[Dict[str, Any]]) -> Dict[str, Any]:
    user = USER_TEMPLATE.format(blocks_json=json.dumps(blocks_chunk, ensure_ascii=False))
    headers = {
        "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}",
        "Content-Type": "application/json",
    }
    body = {
        "model": MODEL_NAME,
        "temperature": TEMPERATURE,
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user",   "content": user},
        ],
        "response_format": {"type": "json_object"},
        "max_tokens": 2048,
    }
    r = requests.post(API_URL, headers=headers, json=body, timeout=60)
    if r.status_code != 200:
        raise RuntimeError(f"OpenAI API error {r.status_code}: {r.text[:500]}")
    data = r.json()
    txt = data["choices"][0]["message"]["content"]
    # Strip optional code fences if any
    if txt and txt.strip().startswith("```"):
        t = txt.strip().strip("`")
        if t.lower().startswith("json"):
            t = t[4:].strip()
        txt = t
    return json.loads(txt)

# G) Batch runner
def run_batches(df: pd.DataFrame) -> List[Dict[str, Any]]:
    out = []
    n = len(df)
    if MAX_BLOCKS is not None:
        df = df.head(MAX_BLOCKS); n = len(df)
    for i in range(0, n, BATCH_SIZE):
        chunk_df = df.iloc[i:i+BATCH_SIZE]
        payload = make_blocks_payload(chunk_df)
        tries = 0
        while True:
            try:
                out.append(call_llm(payload))
                break
            except Exception as e:
                tries += 1
                if tries >= RETRY:
                    print(f"[WARN] batch {i}-{i+len(chunk_df)} failed: {e}")
                    out.append({"results":[]})
                    break
                time.sleep(SLEEP_BETWEEN * tries)
    return out

#H) Validate & normalise
def validate_and_flatten(all_responses: List[Dict[str, Any]]) -> pd.DataFrame:
    facts: List[Dict[str, Any]] = []
    for resp in all_responses:
        results = resp.get("results", [])
        if not isinstance(results, list):
            continue
        for obj in results:
            try:
                if "unit" in obj: obj["unit"] = normalise_unit(obj["unit"])
                if "error_unit" in obj: obj["error_unit"] = normalise_unit(obj["error_unit"])
                fact = Fact(**obj)
                facts.append(fact.dict())
            except ValidationError:
                safe = {k:v for k,v in obj.items() if k in Fact.model_fields}
                try:
                    if "unit" in safe: safe["unit"] = normalise_unit(safe["unit"])
                    if "error_unit" in safe: safe["error_unit"] = normalise_unit(safe["error_unit"])
                    fact = Fact(**safe)
                    facts.append(fact.dict())
                except Exception:
                    pass

    if not facts:
        return pd.DataFrame(columns=list(Fact.model_fields.keys()))

    df = pd.DataFrame(facts)
    df["confidence"] = df["confidence"].clip(0.0, 1.0).fillna(0.5)
    df["category"] = df["category"].str.strip().str.lower().map({
        "lithology":"lithology","rock_type":"rock_type","minerals":"minerals",
        "mineralisation":"mineralisation","tectonism_event":"tectonism_event",
        "structures":"structures","assay":"assay","geochemistry":"geochemistry",
        "geochronology":"geochronology","magmatism":"magmatism","methods":"methods",
        "stratigraphy":"stratigraphy"
    }).fillna("assay")

    for col in ["value","value_min","value_max","error"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    return df.sort_values(["page","block_id","category","attribute"]).reset_index(drop=True)


cands_df = df_cands[["page_num","block_id","text"]]
responses = run_batches(cands_df)
df_facts   = validate_and_flatten(responses)


JSONL_PATH = "/content/extracted_step4.jsonl"
CSV_PATH   = "/content/extracted_step4.csv"
PARQ_PATH  = "/content/extracted_step4.parquet"

with open(JSONL_PATH, "w", encoding="utf-8") as f:
    for _, r in df_facts.iterrows():
        f.write(json.dumps(r.to_dict(), ensure_ascii=False) + "\n")

df_facts.to_csv(CSV_PATH, index=False)
try:
    df_facts.to_parquet(PARQ_PATH, index=False)
except Exception:
    pass

print(f"Saved JSONL → {JSONL_PATH}")
print(f"Saved CSV   → {CSV_PATH}")
print(f"Saved Parquet (if available) → {PARQ_PATH}")
print("\nPreview:")
print(df_facts.head(10).to_string(index=False))
print("\nCounts by category:")
print(df_facts["category"].value_counts())


Saved JSONL → /content/extracted_step4.jsonl
Saved CSV   → /content/extracted_step4.csv
Saved Parquet (if available) → /content/extracted_step4.parquet

Preview:
     category entity attribute  value unit  value_min  value_max  error error_unit method mineral_phase sample_id                                                                        context_snippet  page block_id  confidence
geochronology   None       age 2158.0   Ma        NaN        NaN    NaN       None   None          None      None 2158 Ma gold at Wassa, Parra-Avila, in press; 2105 ± 2 Ma gold at Ashanti, Oberthür et    19    19-18         0.9
geochronology   None       age 2105.0   Ma        NaN        NaN    2.0       None   None          None      None 2158 Ma gold at Wassa, Parra-Avila, in press; 2105 ± 2 Ma gold at Ashanti, Oberthür et    19    19-18         0.9
geochronology   None       age 2063.0   Ma        NaN        NaN    9.0       None   None          None      None               2063 ± 9 Ma gold at Damang

/tmp/ipython-input-724341226.py:186: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  facts.append(fact.dict())


In [ ]:

df = df_facts.copy()

# fill missing error_unit when unit exists and error is present
mask_err_no_unit = df["error"].notna() & df["unit"].notna() & df["error_unit"].isna()
df.loc[mask_err_no_unit, "error_unit"] = df.loc[mask_err_no_unit, "unit"]

# infer unit for ranges from context
def infer_unit_from_context(snippet, current):
    if current: return current
    s = (snippet or "").lower()
    if " ma" in s: return "Ma"
    if " ga" in s: return "Ga"
    if " g/t" in s: return "g/t"
    if " ppm" in s: return "ppm"
    if " wt%" in s or " wt %" in s or " %" in s: return "%"
    return None

rng = df["value"].isna() & df["value_min"].notna() & df["value_max"].notna()
df.loc[rng, "unit"] = [
    infer_unit_from_context(sn, u) for sn, u in zip(df.loc[rng, "context_snippet"], df.loc[rng, "unit"])
]

# basic dedupe: keep max-confidence per key
keys = ["page","block_id","category","attribute","value","value_min","value_max","unit","error","error_unit"]
df = (df
      .sort_values("confidence", ascending=False)
      .drop_duplicates(subset=keys, keep="first")
      .reset_index(drop=True))

df.to_csv("/content/extracted_step4_clean.csv", index=False)
print("Saved cleaned CSV → /content/extracted_step4_clean.csv")
print(df.head(10).to_string(index=False))


Saved cleaned CSV → /content/extracted_step4_clean.csv
Empty DataFrame
Columns: [category, entity, attribute, value, unit, value_min, value_max, error, error_unit, method, mineral_phase, sample_id, context_snippet, page, block_id, confidence]
Index: []


# step 5


In [ ]:

import os, json
import pandas as pd

#A) Load input (change if needed)
PREFERRED = "/content/extracted_step4_clean.csv"
FALLBACKS = ["/content/extracted_step4.csv", "/content/extracted_step4_offline.csv"]

def load_step4(path=PREFERRED, fallbacks=FALLBACKS):
    if os.path.exists(path):
        return pd.read_csv(path)
    for fb in fallbacks:
        if os.path.exists(fb):
            print(f"[Note] Using fallback: {fb}")
            return pd.read_csv(fb)
    raise FileNotFoundError("No Step-4 CSV found. Expected one of: "
                            f"{[path]+fallbacks}")

df = load_step4()
print(f"Loaded Step-4 facts: {len(df)} rows, columns={list(df.columns)}")


# Coerce numerics; normalize unit strings a bit (optional tweak)
for col in ["value","value_min","value_max","error","confidence","page"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

def norm_unit(u):
    if pd.isna(u): return None
    s = str(u).strip().replace("ug/g","µg/g")
    mapc = {"ma":"Ma","ga":"Ga","°c":"°C"}
    return mapc.get(s.lower(), s)

if "unit" in df.columns:
    df["unit"] = df["unit"].apply(norm_unit)
if "error_unit" in df.columns:
    df["error_unit"] = df["error_unit"].apply(norm_unit)

# Ensure essential columns exist
for col in ["category","attribute","context_snippet","page","block_id","confidence"]:
    if col not in df.columns:
        df[col] = None

# Summary pivot
summary_by_cat = df["category"].value_counts().sort_values(ascending=False)
summary_by_page = df.groupby("page")["category"].count().sort_index()

print("\n=== Step 5: Summary ===")
print("By category:\n", summary_by_cat.to_string())
print("\nBy page (non-zero only):\n", summary_by_page[summary_by_page>0].head(20).to_string())


OUT_DIR = "/content/step5_categories"
os.makedirs(OUT_DIR, exist_ok=True)

cat_paths = {}
for cat, sub in df.groupby(df["category"].fillna("uncategorized")):
    path = os.path.join(OUT_DIR, f"{cat}.csv")
    sub.sort_values(["page","block_id","attribute","value"], inplace=True, kind="stable")
    sub.to_csv(path, index=False)
    cat_paths[cat] = path

print(f"\nSaved per-category CSVs → {OUT_DIR}")
for k, v in list(cat_paths.items())[:8]:
    print(f"  {k:16s} → {v}")

# Lookup tables
# 1) Ages (geochronology)
ages = df[df["category"]=="geochronology"].copy()
# prefer scalar 'value' else range min/max
ages["age_value"] = ages["value"]
ages["age_min"]   = ages["value_min"]
ages["age_max"]   = ages["value_max"]
ages_lookup = (ages[[
    "page","block_id","attribute","age_value","age_min","age_max",
    "error","unit","error_unit","method","mineral_phase","sample_id",
    "context_snippet","confidence"
]]
    .sort_values(["page","block_id","age_value","age_min","age_max","confidence"], ascending=[True,True,True,True,True,False])
    .reset_index(drop=True)
)
AGES_PATH = "/content/geochronology_ages.csv"
ages_lookup.to_csv(AGES_PATH, index=False)

assay = df[df["category"]=="assay"].copy()
assay_lookup = (assay[[
    "page","block_id","attribute","value","value_min","value_max","unit",
    "method","mineral_phase","sample_id","context_snippet","confidence"
]]
    .sort_values(["attribute","page","block_id","value","value_min","value_max","confidence"], ascending=[True,True,True,True,True,True,False])
    .reset_index(drop=True)
)
ASSAY_PATH = "/content/assay_grades.csv"
assay_lookup.to_csv(ASSAY_PATH, index=False)

print(f"\nSaved lookups:\n  Ages → {AGES_PATH}\n  Assays → {ASSAY_PATH}")


CATALOG_PATH = "/content/step5_catalog.jsonl"
with open(CATALOG_PATH, "w", encoding="utf-8") as f:
    for _, r in df.iterrows():

        rec = {
            "category": r.get("category"),
            "attribute": r.get("attribute"),
            "unit": r.get("unit"),
            "value": r.get("value"),
            "value_min": r.get("value_min"),
            "value_max": r.get("value_max"),
            "error": r.get("error"),
            "page": int(r.get("page")) if pd.notna(r.get("page")) else None,
            "block_id": r.get("block_id"),
            "confidence": r.get("confidence"),
            "context_snippet": r.get("context_snippet"),
        }
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")
print(f"Saved catalog JSONL → {CATALOG_PATH}")


def find_facts(query: str, category: str=None, attribute: str=None, top: int=30):
    """Keyword search across context + attribute with optional filters."""
    q = str(query).strip().lower()
    sub = df.copy()
    if category:
        sub = sub[sub["category"].str.lower()==category.lower()]
    if attribute:
        sub = sub[sub["attribute"].fillna("").str.lower()==attribute.lower()]
    mask = (
        sub["context_snippet"].fillna("").str.lower().str.contains(q, na=False) |
        sub["attribute"].fillna("").str.lower().str.contains(q, na=False)
    )
    out = (sub[mask]
           .sort_values(["confidence","page"], ascending=[False,True])
           .head(top)
           .reset_index(drop=True))
    return out[[
        "category","attribute","value","value_min","value_max","unit",
        "error","error_unit","page","block_id","confidence","context_snippet"
    ]]


print("\nSample search: 'Au' (assay)")
print(find_facts("Au", category="assay").head(8).to_string(index=False))

print("\nSample search: '±' (ages with errors)")
print(find_facts("±", category="geochronology").head(8).to_string(index=False))

# Final pointers
print("\nArtifacts:")
print(f"  Per-category dir: {OUT_DIR}")
print(f"  Ages lookup:      {AGES_PATH}")
print(f"  Assay lookup:     {ASSAY_PATH}")
print(f"  Catalog JSONL:    {CATALOG_PATH}")


Loaded Step-4 facts: 0 rows, columns=['category', 'entity', 'attribute', 'value', 'unit', 'value_min', 'value_max', 'error', 'error_unit', 'method', 'mineral_phase', 'sample_id', 'context_snippet', 'page', 'block_id', 'confidence']

=== Step 5: Summary ===
By category:
 Series([], )

By page (non-zero only):
 Series([], )

Saved per-category CSVs → /content/step5_categories

Saved lookups:
  Ages → /content/geochronology_ages.csv
  Assays → /content/assay_grades.csv
Saved catalog JSONL → /content/step5_catalog.jsonl

Sample search: 'Au' (assay)
Empty DataFrame
Columns: [category, attribute, value, value_min, value_max, unit, error, error_unit, page, block_id, confidence, context_snippet]
Index: []

Sample search: '±' (ages with errors)
Empty DataFrame
Columns: [category, attribute, value, value_min, value_max, unit, error, error_unit, page, block_id, confidence, context_snippet]
Index: []

Artifacts:
  Per-category dir: /content/step5_categories
  Ages lookup:      /content/geochronolo

In [ ]:
#Step 8) Referencing & Provenance Attachment
!pip -q install pypdf pandas

import re, json, os
import pandas as pd
from pathlib import Path
from pypdf import PdfReader


FACTS_CSV = "/content/extracted_step4_clean.csv"
PDF_PATH  = "/content/PhDThesis__Masurel2015.pdf"  # same thesis used in Step 1

assert os.path.exists(FACTS_CSV), f"Missing {FACTS_CSV}"
assert os.path.exists(PDF_PATH),  f"Missing {PDF_PATH}"

df = pd.read_csv(FACTS_CSV)


def read_pdf_pages_text(pdf_path: str):
    reader = PdfReader(pdf_path)
    out = []
    for i, page in enumerate(reader.pages):
        try:
            txt = page.extract_text() or ""
        except Exception:
            txt = ""
        out.append(txt)
    return out

pages_text = read_pdf_pages_text(PDF_PATH)
N = len(pages_text)


TITLE_PATS = [
    r'^\s*(chapter\s+[ivx\d]+\.?\s+.*)$',     # Chapter I / Chapter 1 ...
    r'^\s*chapter\s+.*$',                     # "Chapter ..." loose
    r'^\s*\d+\.\s+[A-Z].{3,}$',               # "1. Title"
    r'^\s*[A-Z][A-Za-z0-9\-\s]{4,}$',         # ALL/Title-like single line
]
TITLE_REGEX = re.compile("|".join(TITLE_PATS), re.IGNORECASE | re.MULTILINE)

def guess_section_title(page_text: str) -> str|None:
    head = page_text.strip().splitlines()

    for line in head[:25]:
        m = TITLE_REGEX.search(line.strip())
        if m:

            if m.lastindex and m.group(m.lastindex):
                return m.group(m.lastindex).strip()
            return line.strip()
    return None


CITE_RE = re.compile(
    r'\(([A-Z][A-Za-z\-]+(?:\s*&\s*[A-Z][A-Za-z\-]+)?|[A-Z][A-Za-z\-]+ et al\.)\s*,\s*(\d{4}[a-z]?)\)',
    re.UNICODE
)

def extract_citations(text: str, limit=6):
    cits = [f"{a} {y}" for a, y in CITE_RE.findall(text)]
    # de-dup, preserve order
    seen, uniq = set(), []
    for c in cits:
        if c not in seen:
            uniq.append(c); seen.add(c)
    return uniq[:limit]


provenance_rows = []
for i, r in df.iterrows():
    page = int(r.get("page", 0))
    pg_idx = max(0, min(N-1, page-1))

    page_text = pages_text[pg_idx] if 0 <= pg_idx < N else ""
    section_title = guess_section_title(page_text) or ""


    context = str(r.get("context_snippet") or "")
    cites_ctx = extract_citations(context)
    cites_pg  = extract_citations(page_text)
    local_citations = list(dict.fromkeys(cites_ctx + cites_pg))

    provenance_id = f"{page}:{r.get('block_id')}"
    pdf_page_hint = f"#page={page}"
    provenance_note = f"thesis page {page}, block {r.get('block_id')}"

    provenance_rows.append({
        "provenance_id": provenance_id,
        "pdf_page_hint": pdf_page_hint,
        "section_title": section_title,
        "local_citations": "; ".join(local_citations) if local_citations else "",
        "provenance_note": provenance_note
    })

prov = pd.DataFrame(provenance_rows, index=df.index)

# Merge & save
df_enriched = pd.concat([df, prov], axis=1)

OUT_ENRICHED = "/content/extracted_step8_provenance.csv"
df_enriched.to_csv(OUT_ENRICHED, index=False)
print(f"Saved with provenance → {OUT_ENRICHED}")


show_cols = ["category","attribute","value","value_min","value_max","unit","error","page","block_id","section_title","local_citations","provenance_id"]
print(df_enriched[show_cols].head(10).to_string(index=False))


Saved with provenance → /content/extracted_step8_provenance.csv


KeyError: "['section_title', 'local_citations', 'provenance_id'] not in index"

step 9 output writers ( master table + category sheets)

In [ ]:

import os, re, json, uuid, hashlib
from pathlib import Path
import pandas as pd
import numpy as np


FACTS_IN   = "/content/extracted_step8_provenance.csv"   # from Step 8
PDF_PATH   = "/content/PhDThesis__Masurel2015.pdf"       # same as Step 1
THESIS_ID  = "Masurel2015"
PIPE_VER   = "v0.9-step9"

assert os.path.exists(FACTS_IN), f"Missing {FACTS_IN}"

df = pd.read_csv(FACTS_IN)
print(f"Loaded Step-8 enriched facts: {len(df)} rows")


ALLOWED_CATEGORIES = {
    "geochronology","assay","geochemistry","lithology","minerals",
    "mineralisation","structures","tectonism_event","methods","stratigraphy","rock_type"
}
UNIT_MAP_LOWER = {"ma":"Ma","ga":"Ga","°c":"°C","ug/g":"µg/g","percent":"%","gpt":"g/t"}
def norm_unit(u):
    if pd.isna(u): return None
    s = str(u).strip()
    s2 = UNIT_MAP_LOWER.get(s.lower(), s)
    return s2

def norm_category(c):
    if pd.isna(c): return "assay"
    s = str(c).strip().lower().replace("–","-")
    # canonical map
    m = {
        "geochronology":"geochronology","assay":"assay","geochemistry":"geochemistry",
        "lithology":"lithology","minerals":"minerals","mineralisation":"mineralisation",
        "mineralization":"mineralisation","structures":"structures",
        "tectonism_event":"tectonism_event","methods":"methods",
        "stratigraphy":"stratigraphy","rock_type":"rock_type"
    }
    return m.get(s, "assay")

for c in ["value","value_min","value_max","error","confidence","page"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

if "unit" in df.columns:       df["unit"] = df["unit"].apply(norm_unit)
if "error_unit" in df.columns: df["error_unit"] = df["error_unit"].apply(norm_unit)
if "category" in df.columns:   df["category"] = df["category"].apply(norm_category)


def make_stable_id(row):
    key = "|".join([
        str(row.get("category", "")),
        str(row.get("attribute","")),
        str(row.get("value","")),
        str(row.get("value_min","")),
        str(row.get("value_max","")),
        str(row.get("unit","")),
        str(row.get("error","")),
        str(row.get("page","")),
        str(row.get("block_id","")),
    ])
    return str(uuid.uuid5(uuid.NAMESPACE_URL, key))

df["fact_id"]    = df.apply(make_stable_id, axis=1)
df["thesis_id"]  = THESIS_ID
df["pdf_path"]   = PDF_PATH if os.path.exists(PDF_PATH) else ""
df["pipeline_version"] = PIPE_VER


def quality_flag(row):
    has_scalar = pd.notna(row.get("value"))
    has_range  = pd.notna(row.get("value_min")) or pd.notna(row.get("value_max"))
    has_num    = bool(has_scalar or has_range or pd.notna(row.get("error")))
    unit       = row.get("unit")
    cat        = row.get("category","")
    method     = str(row.get("method") or "").strip()
    prov_ok    = str(row.get("provenance_id") or "")!=""
    # Rules (worst-first)
    if not has_num:
        return "warn_incomplete"
    if cat in ("assay","geochemistry") and not unit:
        return "warn_missing_unit"
    if cat=="geochronology":
        # ages should carry unit Ma/Ga
        if not unit:
            return "warn_missing_unit"
        if method=="":
            return "warn_missing_method"
    if not prov_ok:
        return "warn_missing_provenance"
    return "ok"

df["quality_flag"] = df.apply(quality_flag, axis=1)


MASTER_COLS = [
    "fact_id","thesis_id","pdf_path","pipeline_version",
    "category","entity","attribute",
    "value","value_min","value_max","unit","error","error_unit",
    "method","mineral_phase","sample_id",
    "page","block_id","section_title","local_citations","context_snippet",
    "provenance_id","quality_flag","confidence"
]
for col in MASTER_COLS:
    if col not in df.columns:
        df[col] = None
master = df[MASTER_COLS].copy()


OUT_DIR = "/content/step9_outputs"
os.makedirs(OUT_DIR, exist_ok=True)
MASTER_CSV  = f"{OUT_DIR}/master_facts.csv"
MASTER_PARQ = f"{OUT_DIR}/master_facts.parquet"
master.to_csv(MASTER_CSV, index=False)
try:
    master.to_parquet(MASTER_PARQ, index=False)
except Exception:
    pass

print(f"Saved Master CSV   → {MASTER_CSV}")
print(f"Saved Master PARQ  → {MASTER_PARQ}")


!pip install -q xlsxwriter
XLSX_PATH = f"{OUT_DIR}/Geo_Master_and_Categories.xlsx"
with pd.ExcelWriter(XLSX_PATH, engine="xlsxwriter") as xw:
    ...


XLSX_PATH = f"{OUT_DIR}/Geo_Master_and_Categories.xlsx"
with pd.ExcelWriter(XLSX_PATH, engine="xlsxwriter") as xw:
    # Overview
    ov = (master["category"].value_counts()
            .rename_axis("category")
            .reset_index(name="count")
            .sort_values("count", ascending=False))
    ov["percent"] = (ov["count"]/len(master)*100).round(1)
    qf = (master["quality_flag"].value_counts()
            .rename_axis("quality_flag")
            .reset_index(name="count")
            .sort_values("count", ascending=False))
    ov.to_excel(xw, sheet_name="Overview", index=False)
    qf.to_excel(xw, sheet_name="Quality",  index=False)

    # Per-category sheets
    order = ["geochronology","assay","geochemistry","lithology","minerals",
             "mineralisation","structures","tectonism_event","methods","stratigraphy","rock_type"]
    cats = sorted(master["category"].dropna().unique(), key=lambda c: (order.index(c) if c in order else 999, c))
    for cat in cats:
        sub = master[master["category"]==cat].copy()
        sub = sub.sort_values(["page","block_id","attribute","value","value_min","value_max","confidence"],
                              ascending=[True,True,True,True,True,True,False])
        # keep most useful columns for browsing
        view_cols = [
            "fact_id","attribute","value","value_min","value_max","unit","error","error_unit",
            "method","mineral_phase","sample_id","page","block_id","section_title",
            "local_citations","quality_flag","confidence","context_snippet"
        ]
        sub[view_cols].to_excel(xw, sheet_name=cat[:31], index=False)

print(f"Saved Excel (master + sheets) → {XLSX_PATH}")


print("\n=== Step 9 Summary ===")
print("By category:\n", master["category"].value_counts().to_string())
print("\nBy quality_flag:\n", master["quality_flag"].value_counts().to_string())

bad = master[master["quality_flag"]!="ok"].head(12)
if len(bad):
    print("\nSample non-OK rows:\n", bad[["category","attribute","value","unit","method","page","block_id","quality_flag"]].to_string(index=False))


Loaded Step-8 enriched facts: 0 rows
Saved Master CSV   → /content/step9_outputs/master_facts.csv
Saved Master PARQ  → /content/step9_outputs/master_facts.parquet
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 11.5 MB/s eta 0:00:00
Saved Excel (master + sheets) → /content/step9_outputs/Geo_Master_and_Categories.xlsx

=== Step 9 Summary ===
By category:
 Series([], )

By quality_flag:
 Series([], )


In [ ]:

import os, re, uuid
import pandas as pd
from pathlib import Path

MASTER_IN   = "/content/step9_outputs/master_facts.csv"     # from Step 9
PDF_PATH    = "/content/PhDThesis__Masurel2015.pdf"         # optional (helps page-level unit inference)
OUT_DIR     = "/content/step9_outputs"
XLSX_PATH   = f"{OUT_DIR}/Geo_Master_and_Categories.xlsx"

assert os.path.exists(MASTER_IN), f"Missing {MASTER_IN}"
df = pd.read_csv(MASTER_IN)


pages_text = None
if os.path.exists(PDF_PATH):
    try:
        from pypdf import PdfReader
        pages_text = []
        rdr = PdfReader(PDF_PATH)
        for p in rdr.pages:
            try:
                pages_text.append(p.extract_text() or "")
            except Exception:
                pages_text.append("")
        print(f"[info] Loaded {len(pages_text)} pages for backfill context.")
    except Exception as e:
        print("[warn] Could not load PDF for page text:", e)

def page_text(page_1based:int) -> str:
    if pages_text is None: return ""
    i = max(0, min(len(pages_text)-1, int(page_1based)-1))
    return pages_text[i]


UNIT_CUES = {
    "Ma": re.compile(r'\bMa\b', re.I),
    "Ga": re.compile(r'\bGa\b', re.I),
    "ppm": re.compile(r'\bppm\b', re.I),
    "ppb": re.compile(r'\bppb\b', re.I),
    "g/t": re.compile(r'\bg\/t\b', re.I),
    "%":   re.compile(r'(?<!wt)\b%\b', re.I),
    "wt%": re.compile(r'\bwt%\b', re.I),
}
METHOD_CUES = {
    "U–Pb":   re.compile(r'\bU[–-]Pb\b', re.I),
    "Pb–Pb":  re.compile(r'\bPb[–-]Pb\b', re.I),
    "Sm–Nd":  re.compile(r'\bSm[–-]Nd\b', re.I),
    "Ar–Ar":  re.compile(r'\bAr[–-]Ar\b', re.I),
    "Rb–Sr":  re.compile(r'\bRb[–-]Sr\b', re.I),
    "Lu–Hf":  re.compile(r'\bLu[–-]Hf\b', re.I),
    "Re–Os":  re.compile(r'\bRe[–-]Os\b', re.I),
}
PHASE_CUES = {
    "zircon": re.compile(r'\bzircon\b', re.I),
    "monazite": re.compile(r'\bmonazite\b', re.I),
    "baddeleyite": re.compile(r'\bbaddeleyite\b', re.I),
}

def infer_from_text(text: str, cues: dict[str, re.Pattern]) -> list[str]:
    hits = []
    low = text or ""
    for key, pat in cues.items():
        if pat.search(low):
            hits.append(key)
    return hits


before_missing_units = (df["unit"].isna() | (df["unit"].astype(str).str.strip()=="")).sum()

for i, r in df.iterrows():
    cat   = str(r.get("category","")).lower().strip()
    unit  = (None if pd.isna(r.get("unit")) else str(r.get("unit")).strip()) or None
    meth  = (None if pd.isna(r.get("method")) else str(r.get("method")).strip()) or None
    phase = (None if pd.isna(r.get("mineral_phase")) else str(r.get("mineral_phase")).strip()) or None

    # Build context (snippet + page)
    ctx = str(r.get("context_snippet") or "")
    pg  = int(r.get("page") or 0)
    full = (ctx + " " + page_text(pg)).strip()

    # 1) Unit backfill
    if unit is None:
        # geochronology first: look for Ma/Ga
        if cat == "geochronology":
            if UNIT_CUES["Ma"].search(full):
                unit = "Ma"
            elif UNIT_CUES["Ga"].search(full):
                unit = "Ga"
        # assays/geochem
        if unit is None:
            for key in ["g/t","ppm","ppb","wt%","%"]:
                if UNIT_CUES[key].search(full):
                    unit = key
                    break
        if unit is not None:
            df.at[i, "unit"] = unit

    # 2) Method backfill (if obvious)
    if not meth:
        hits = infer_from_text(full, METHOD_CUES)
        if hits:
            df.at[i, "method"] = hits[0]  # first strongest

    # 3) Mineral phase backfill (if obvious)
    if not phase and cat == "geochronology":
        ph = infer_from_text(full, PHASE_CUES)
        if ph:
            df.at[i, "mineral_phase"] = ph[0]

after_missing_units = (df["unit"].isna() | (df["unit"].astype(str).str.strip()=="")).sum()
print(f"Units filled: {before_missing_units - after_missing_units} rows")


def recompute_qf(row):
    has_scalar = pd.notna(row.get("value"))
    has_range  = pd.notna(row.get("value_min")) or pd.notna(row.get("value_max"))
    has_num    = bool(has_scalar or has_range or pd.notna(row.get("error")))
    unit       = (None if pd.isna(row.get("unit")) else str(row.get("unit")).strip()) or None
    cat        = str(row.get("category",""))
    method     = str(row.get("method") or "").strip()
    prov_ok    = str(row.get("provenance_id") or "")!=""
    if not has_num:
        return "warn_incomplete"
    if cat in ("assay","geochemistry") and not unit:
        return "warn_missing_unit"
    if cat=="geochronology":
        if not unit:
            return "warn_missing_unit"
        if method=="":
            return "warn_missing_method"
    if not prov_ok:
        return "warn_missing_provenance"
    return "ok"

df["quality_flag"] = df.apply(recompute_qf, axis=1)


MASTER_CSV  = f"{OUT_DIR}/master_facts.csv"
MASTER_PARQ = f"{OUT_DIR}/master_facts.parquet"
df.to_csv(MASTER_CSV, index=False)
try:
    df.to_parquet(MASTER_PARQ, index=False)
except Exception:
    pass
print("Re-saved Master CSV/PARQ with backfills.")

# Excel rewrite
!pip -q install xlsxwriter
with pd.ExcelWriter(XLSX_PATH, engine="xlsxwriter") as xw:
    ov = (df["category"].value_counts()
          .rename_axis("category")
          .reset_index(name="count")
          .sort_values("count", ascending=False))
    ov["percent"] = (ov["count"]/len(df)*100).round(1)
    qf = (df["quality_flag"].value_counts()
          .rename_axis("quality_flag")
          .reset_index(name="count")
          .sort_values("count", ascending=False))
    ov.to_excel(xw, sheet_name="Overview", index=False)
    qf.to_excel(xw, sheet_name="Quality",  index=False)

    order = ["geochronology","assay","geochemistry","lithology","minerals",
             "mineralisation","structures","tectonism_event","methods","stratigraphy","rock_type"]
    cats = sorted(df["category"].dropna().unique(), key=lambda c: (order.index(c) if c in order else 999, c))
    for cat in cats:
        sub = df[df["category"]==cat].copy()
        sub = sub.sort_values(["page","block_id","attribute","value","value_min","value_max","confidence"],
                              ascending=[True,True,True,True,True,True,False])
        cols = [
            "fact_id","attribute","value","value_min","value_max","unit","error","error_unit",
            "method","mineral_phase","sample_id","page","block_id","section_title",
            "local_citations","quality_flag","confidence","context_snippet"
        ]
        cols = [c for c in cols if c in sub.columns]
        sub[cols].to_excel(xw, sheet_name=cat[:31], index=False)

print("Updated Excel written.")
